In [1]:
import torch

#全局变量
hub_token =None # open('/root/hub_token.txt').read().strip()
repo_id = 'BirdL/DALL-E-Cats'#'BirdL/DALL-E-Dogs'#'BirdL/DALL-E-Cats'#'lansinuote/diffusion.1.unconditional'ULZIITOGTOKH/cat_images
push_to_hub =False#True

In [2]:
from datasets import load_dataset


def get_dataset():
    #加载数据集
    dataset = load_dataset(repo_id)#('BirdL/DALL-E-Cats')#('huggan/flowers-102-categories', download=True)
    #dataset = load_dataset('huggan/pokemon') 

    return dataset


In [3]:
dataset = load_dataset(path=repo_id, split='train')


dataset, dataset[0]

(Dataset({
     features: ['Images'],
     num_rows: 300
 }),
 {'Images': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=256x256>})

In [4]:
from datasets import load_dataset

# 加载数据集
dataset = load_dataset(path=repo_id, split='train')

# 查看数据集的特征信息
print(dataset.features)



{'Images': Image(decode=True, id=None)}


In [5]:
import torchvision

#图像增强
compose = torchvision.transforms.Compose([
    torchvision.transforms.Resize(
       128, interpolation=torchvision.transforms.InterpolationMode.BILINEAR),#64  128  32
    torchvision.transforms.RandomCrop(128),
    #torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.5], [0.5])
])


def f(data):
    image = [compose(i) for i in data['Images']]
    return {'Images': image}# return {'image': image}


#因为图像增强在每个epoch中要动态计算,所以不能简单地用map处理
dataset.set_transform(f)

dataset, dataset[0]

(Dataset({
     features: ['Images'],
     num_rows: 300
 }),
 {'Images': tensor([[[-0.6471, -0.6471, -0.6392,  ..., -0.6941, -0.6941, -0.7020],
           [-0.6314, -0.6392, -0.6314,  ..., -0.6941, -0.6941, -0.6941],
           [-0.6157, -0.6314, -0.6235,  ..., -0.6863, -0.6863, -0.6941],
           ...,
           [-0.8118, -0.8118, -0.8039,  ..., -0.7647, -0.7725, -0.7725],
           [-0.8118, -0.8118, -0.8118,  ..., -0.7725, -0.7647, -0.7647],
           [-0.8275, -0.8353, -0.8196,  ..., -0.7804, -0.7804, -0.7725]],
  
          [[-0.6863, -0.6784, -0.6784,  ..., -0.7176, -0.7255, -0.7176],
           [-0.6627, -0.6627, -0.6706,  ..., -0.7176, -0.7098, -0.7020],
           [-0.6471, -0.6549, -0.6549,  ..., -0.7176, -0.7098, -0.7098],
           ...,
           [-0.8196, -0.8118, -0.8196,  ..., -0.7804, -0.7804, -0.7804],
           [-0.8196, -0.8196, -0.8196,  ..., -0.7804, -0.7804, -0.7804],
           [-0.8431, -0.8275, -0.8275,  ..., -0.7961, -0.7882, -0.7804]],
  
          [[

In [6]:
loader = torch.utils.data.DataLoader(dataset,
                                     batch_size=8,#16  16（50，1280128）32（50 128 128）
                                     shuffle=True,
                                     drop_last=False)

len(loader), next(iter(loader))

(38,
 {'Images': tensor([[[[-1.0000, -1.0000, -0.9922,  ..., -0.9765, -0.9765, -0.9765],
            [-1.0000, -1.0000, -1.0000,  ..., -0.9843, -0.9765, -0.9765],
            [-1.0000, -0.9922, -0.9922,  ..., -0.9843, -0.9765, -0.9765],
            ...,
            [-1.0000, -1.0000, -1.0000,  ...,  0.0039, -0.0196, -0.0353],
            [-1.0000, -1.0000, -1.0000,  ...,  0.0275, -0.0196, -0.0275],
            [-1.0000, -1.0000, -1.0000,  ...,  0.0039, -0.0353, -0.0039]],
  
           [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
            [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
            [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
            ...,
            [-1.0000, -1.0000, -1.0000,  ..., -0.0353, -0.0588, -0.0745],
            [-1.0000, -1.0000, -1.0000,  ..., -0.0196, -0.0745, -0.0980],
            [-1.0000, -1.0000, -1.0000,  ..., -0.0588, -0.0824, -0.0588]],
  
           [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, 

In [7]:
from diffusers import UNet2DModel

#定义模型,随机初始化参数
model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=3,#初始是2
    block_out_channels=(128, 128, 256, 256, 512, 512),
    down_block_types=(
        'DownBlock2D',
        'DownBlock2D',
        'DownBlock2D',
        'DownBlock2D',
        'AttnDownBlock2D',
        'DownBlock2D',
    ),
    up_block_types=(
        'UpBlock2D',
        'AttnUpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
        'UpBlock2D',
    ),
)

sum(i.numel() for i in model.parameters()) / 10000

14998.4003

In [8]:
from diffusers import DDPMScheduler
from diffusers.optimization import get_scheduler

#初始化工具类
scheduler = DDPMScheduler(num_train_timesteps=1000,
                          beta_schedule='linear',
                          prediction_type='epsilon')

optimizer = torch.optim.AdamW(model.parameters(),
                              lr=1e-5,#1e-4 1e-5
                              betas=(0.995, 0.999),#0.95
                              weight_decay=1e-6,
                              eps=1e-8)

scheduler_lr = get_scheduler('cosine',
                             optimizer=optimizer,
                             num_warmup_steps=500,
                             num_training_steps=len(loader) * 100)

criterion = torch.nn.MSELoss()

scheduler, optimizer, scheduler_lr, criterion

(DDPMScheduler {
   "_class_name": "DDPMScheduler",
   "_diffusers_version": "0.27.2",
   "beta_end": 0.02,
   "beta_schedule": "linear",
   "beta_start": 0.0001,
   "clip_sample": true,
   "clip_sample_range": 1.0,
   "dynamic_thresholding_ratio": 0.995,
   "num_train_timesteps": 1000,
   "prediction_type": "epsilon",
   "rescale_betas_zero_snr": false,
   "sample_max_value": 1.0,
   "steps_offset": 0,
   "thresholding": false,
   "timestep_spacing": "leading",
   "trained_betas": null,
   "variance_type": "fixed_small"
 },
 AdamW (
 Parameter Group 0
     amsgrad: False
     betas: (0.995, 0.999)
     capturable: False
     eps: 1e-08
     foreach: None
     initial_lr: 1e-05
     lr: 0.0
     maximize: False
     weight_decay: 1e-06
 ),
 MSELoss())

In [9]:
def get_loss(image):
    device = image.device
    
    #随机噪声
    #[b, 3, 64, 64]
    noise = torch.randn(image.shape).to(device)

    #随机b个噪声步数
    #1000 = scheduler.config.num_train_timesteps
    #[b]
    noise_step = torch.randint(0, 1000, (image.shape[0], ),
                               device=device).long()

    #往图片当中添加噪声
    #[b, 3, 64, 64]
    image_noise = scheduler.add_noise(image, noise, noise_step)

    #把图片里的噪声计算出来
    #[b, 3, 64, 64]
    out = model(image_noise, noise_step).sample

    #求mse loss
    return criterion(out, noise)


get_loss(torch.randn(16, 3, 128,128))#6464

tensor(1.1282, grad_fn=<MseLossBackward0>)

In [11]:
from huggingface_hub import Repository, create_repo
from diffusers import DDPMPipeline
from torch.nn.parallel import DataParallel
import os
import torch
import torch

#import torch
#import torch.nn.parallel
# 假设其他必要的库和函数已经导入

def train():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.to(device)

    model.train()
    # 创建一个目录来保存训练结果
    save_dir = './savecatr500'
    os.makedirs(save_dir, exist_ok=True)

    # 创建一个文件用于记录loss
    loss_file = os.path.join(save_dir, 'losses.txt')
    with open(loss_file, 'w') as f:
        f.write('Epoch, Loss\n')  # 写入标题行

    loss_sum = 0
    for epoch in range(500):
        for i, data in enumerate(loader):
            loss = get_loss(data['Images'].to(device))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler_lr.step()
            optimizer.zero_grad()

            loss_sum += loss.item()  # 累积loss

        # 计算平均loss
        average_loss = loss_sum / len(loader)
        loss_sum = 0  # 重置累积loss

        # 打印当前epoch的loss
        print(f'Epoch {epoch}, Loss: {average_loss}')

        # 将当前epoch的loss写入文件
        with open(loss_file, 'a') as f:
            f.write(f'{epoch}, {average_loss:.6f}\n')  # 写入当前epoch和loss

    # 保存模型
    DDPMPipeline(unet=model, scheduler=scheduler).save_pretrained(save_dir)



# 调用训练函数
train()





Epoch 0, Loss: 0.013173873205424139
Epoch 1, Loss: 0.015470477176438036
Epoch 2, Loss: 0.011140411765633249
Epoch 3, Loss: 0.014147320259852629
Epoch 4, Loss: 0.01410327803368043
Epoch 5, Loss: 0.01451598870641503
Epoch 6, Loss: 0.013111798374570514
Epoch 7, Loss: 0.011961815131239985
Epoch 8, Loss: 0.012987377796028005
Epoch 9, Loss: 0.010991242983819623
Epoch 10, Loss: 0.015370951821463868
Epoch 11, Loss: 0.017736459261198576
Epoch 12, Loss: 0.010049318671103959
Epoch 13, Loss: 0.012128549814836955
Epoch 14, Loss: 0.016689623283271334
Epoch 15, Loss: 0.014736769344093964
Epoch 16, Loss: 0.01350845473720447
Epoch 17, Loss: 0.012170238702214863
Epoch 18, Loss: 0.012282445358945742
Epoch 19, Loss: 0.015477474999467009
Epoch 20, Loss: 0.011508478145850333
Epoch 21, Loss: 0.010854914585299986
Epoch 22, Loss: 0.013542265514843166
Epoch 23, Loss: 0.012367168986728709
Epoch 24, Loss: 0.011881040825851653
Epoch 25, Loss: 0.017416678191358715
Epoch 26, Loss: 0.009683870471474764
Epoch 27, Loss